## Human-in-the-Loop
There are two main ways to interact with the team from your application:

1. During a team’s run – execution of run() or run_stream(), provide feedback through a UserProxyAgent.

2. Once the run terminates, provide feedback through input to the next call to run() or run_stream().

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## UserProxyAgent - Providing Feedback During a Run
The UserProxyAgent is a special built-in agent that acts as a proxy for a user to provide feedback to the team.

To use the UserProxyAgent, you can create an instance of it and include it in the team before running the team. The team will decide when to call the UserProxyAgent to ask for feedback from the user.

When UserProxyAgent is called during a run, it blocks the execution of the team until the user provides feedback or errors out. This will hold up the team’s progress and put the team in an unstable state that cannot be saved or resumed.

Due to the blocking nature of this approach, it is recommended to use it only for short interactions that require immediate feedback from the user, such as asking for approval or disapproval with a button click, or an alert requiring immediate attention otherwise failing the task.

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)
# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback.",
)
# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat(
  [primary_agent, critic_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a short poem about the fall season.


/home/vscode/.local/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- primary ----------
Golden leaves in crisp air swaying,
Whispers of autumn softly playing.
Pumpkin patches, fields of gold,
Stories of harvest quietly told.

Sweaters snug, with scarves around,
Every step on crunchy ground.
Sunsets blaze in hues so bright,
Merging day with velvet night.

Chill that dances on the breeze,
Rustling through the ancient trees.
Nature’s lull before the sleep,
Magic moments we all keep.

Time for thanks and firesides,
Embracing cozy as warmth abides.
Fall, a gentle, fleeting queen,
Adorns the earth in colors seen.
---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a smooth, rhythmic flow. The descriptions of golden leaves, pumpkin patches, and cozy attire immediately evoke iconic elements of autumn. Each stanza paints a picture that feels both nostalgic and comforting.

Consider a few suggestions to enhance the poem further:

1. **Add variation in rhythm:** Adding a little variation in t

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=118), content='Golden leaves in crisp air swaying,\nWhispers of autumn softly playing.\nPumpkin patches, fields of gold,\nStories of harvest quietly told.\n\nSweaters snug, with scarves around,\nEvery step on crunchy ground.\nSunsets blaze in hues so bright,\nMerging day with velvet night.\n\nChill that dances on the breeze,\nRustling through the ancient trees.\nNature’s lull before the sleep,\nMagic moments we all keep.\n\nTime for thanks and firesides,\nEmbracing cozy as warmth abides.\nFall, a gentle, fleeting queen,\nAdorns the earth in colors seen.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=353), content="Your poem beautifully captures the essence of the fall season with vivid imag

## Using Max Turns

This method allows you to pause the team for user input by setting a maximum number of turns. 

To implement this, set the max_turns parameter in the RoundRobinGroupChat() constructor.

Once the team stops, the turn count will be reset. 

When you resume the team, it will start from 0 again. However, the team’s internal state will be preserved, for example, the RoundRobinGroupChat will resume from the next agent in the list with the same conversation history.

In [6]:
# Create the team setting a maximum number of turns to 2.
# One chat by agent is one turn. So 2 is needed for the two agents
team = RoundRobinGroupChat([primary_agent, critic_agent], max_turns=2)

task = "Write a short poem about the fall season."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Amber leaves in swirling flight,
Whistling winds through trees take flight.
Pumpkin patches, harvest moon,
Evening shadows coming soon.

Fields aglow with sunset's kiss,
Moments wrapped in autumn's bliss.
Sweaters soft and fires bright,
Kindling warmth in cool twilight.

Apple cider, cinnamon air,
Nature's beauty everywhere.
Whispers from the season's heart,
Fall's embrace—a work of art.
---------- critic ----------
Your poem captures the beauty and essence of the fall season with vivid imagery and a gentle, rhythmic flow. The descriptions of amber leaves, pumpkin patches, and cozy autumn evenings paint a warm and inviting picture. Here are a few suggestions to enhance your poem further:

1. **Sensory Details:** Adding more sensory details can deepen the reader’s immersion in the experience of fall. Mentioning specific scents, sounds, and textures can create a more vivid image.

2. **Cons

## HandoffTermination 
HandoffTermination stops the team when an agent sends a HandoffMessage message.

Note: The model used with AssistantAgent must support tool call to use the handoff feature.

In [7]:
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback first then transfer to user.",
    handoffs=[Handoff(target="user", message="Transfer to user.")],
)

# Define a termination condition that checks for handoff message targetting helper OR text "APPROVE".
handoff_termination = HandoffTermination(target="user")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=handoff_termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in crisp embrace,
Dance their way from tree to space.
Whispers of the autumn breeze,
Rustle secrets through the trees.

Pumpkins rest in fields of gold,
Stories of the harvest told.
Footsteps crunch on paths below,
As nature’s canvas starts to glow.

Sweaters wrapped in cozy hug,
Warmth found in a cider mug.
Skies ablaze in twilight’s hue,
Fall’s brief moments, pure and true.

Fading light as nights extend,
Autumn's beauty soon will end.
But in each leaf’s gentle fall,
Lives the magic of it all.
---------- critic ----------
This poem beautifully captures the essence of the fall season. The imagery of golden leaves, the dance of the autumn breeze, and the warmth found in the simplicity of the season create a vivid and nostalgic scene. The description of pumpkins in fields, the crunch of footsteps, and twilight skies all contribute to the cozy, inviting atmosphere that autumn 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=120), content="Golden leaves in crisp embrace,\nDance their way from tree to space.\nWhispers of the autumn breeze,\nRustle secrets through the trees.\n\nPumpkins rest in fields of gold,\nStories of the harvest told.\nFootsteps crunch on paths below,\nAs nature’s canvas starts to glow.\n\nSweaters wrapped in cozy hug,\nWarmth found in a cider mug.\nSkies ablaze in twilight’s hue,\nFall’s brief moments, pure and true.\n\nFading light as nights extend,\nAutumn's beauty soon will end.\nBut in each leaf’s gentle fall,\nLives the magic of it all.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=182, completion_tokens=106), content='This poem beautifully captures the essence of the fall season. The imagery of golden leav